In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [37]:
import utils
word = 'горшок'
weights = utils.load_weights('../corpora/ad-nouns/cdict/', word)
texts = utils.load_contexts('../corpora/ad-nouns-contexts-100k', word)
texts = [[w for w in ctx if weights.get(w, 0) > 1] for ctx in texts]
texts = [ctx for ctx in texts if ctx]
for text in texts[:3]:
    print(text)
print(len(texts))
#texts = texts[:1000]

['золото', 'почва']
['черепок', 'глиняный']
['орхидея', 'цветок', 'поддон', 'класть', 'гравий', 'сверху', 'ставить', 'орхидея']
32223


In [38]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus[0], corpus[1])

[(0, 1), (1, 1)] [(2, 1), (3, 1)]


In [56]:
#from gensim.models.ldamulticore import LdaMulticore as LdaModel
from gensim.models import LdaModel
from gensim.models import HdpModel
print(len(corpus))
#lda = HdpModel(corpus, id2word=dictionary)
lda = LdaModel(corpus, id2word=dictionary, num_topics=6, passes=3, alpha='auto')

32223


In [52]:
lda.print_topics(num_words=5)
#lda.print_topics(topics=6, topn=5)

[(0, '0.032*детский + 0.022*кувшин + 0.019*см + 0.019*кухня + 0.016*сад'),
 (1,
  '0.103*цветок + 0.093*ребенок + 0.038*ходить + 0.024*золотой + 0.022*приучать'),
 (2,
  '0.100*растение + 0.041*обжигать + 0.032*разбивать + 0.023*корень + 0.021*молоко'),
 (3,
  '0.061*цветочный + 0.058*ночной + 0.044*стоять + 0.037*сидеть + 0.032*окно'),
 (4, '0.037*каша + 0.027*мед + 0.026*глина + 0.020*ложка + 0.017*мясо'),
 (5, '0.063*глиняный + 0.057*вода + 0.028*печь + 0.022*огонь + 0.021*ставить')]

In [53]:
import rl_wsd_labeled
senses, contexts = rl_wsd_labeled.get_contexts(rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', word))
print(senses)
print(contexts[0])

{'2': 'Расширяющийся кверху сосуд с отверстием в дне (цветочный горшок)', '1': 'Округлый глиняный сосуд для приготовления пищи (печной горшок)', '3': 'Ночной горшок'}
(('телевизор, - ковер, , - музыкальный центр, - стол, - аквариум, - 3 шкафа, - цветы в', ' горшках', ', - мелкие аксессуары.'), '2')


In [42]:
print(contexts[0][0])
print(utils.normalize(contexts[0][0]))

('телевизор, - ковер, , - музыкальный центр, - стол, - аквариум, - 3 шкафа, - цветы в', ' горшках', ', - мелкие аксессуары.')
['телевизор', 'ковер', 'музыкальный', 'центр', 'стол', 'аквариум', '3', 'шкаф', 'цветок', 'мелкий', 'аксессуар']


In [54]:
import numpy as np
documents = [dictionary.doc2bow(utils.normalize(ctx)) for ctx, _ in contexts]
gamma, _ = lda.inference(documents)
pred_topics = gamma.argmax(axis=1)
true_labels = np.array([int(ans) for _, ans in contexts])
print(pred_topics[:10])
print(true_labels[:10])

[3 2 3 5 1 3 1 1 0 5]
[2 2 2 2 2 3 2 1 2 2]


In [55]:
from sklearn.metrics import v_measure_score, adjusted_rand_score
print('ARI:', adjusted_rand_score(true_labels, pred_topics))
print('V score:', v_measure_score(true_labels, pred_topics))

ARI: 0.051138243563215696
V score: 0.133119541522
